# dllModel - Python Runner Notebook

![](dllModel.png)

# Python Setup

In [ ]:
import ctypes
from rtwtypes import *

In [ ]:
import os
dll_path = os.path.abspath('first_order_tf_win64.dll')
dll = ctypes.windll.LoadLibrary(dll_path)
# Model entry point functions
model_initialize = dll.first_order_tf_initialize
model_step = dll.first_order_tf_step
model_terminate = dll.first_order_tf_terminate

In [ ]:
InputSignal = real_T.in_dll(dll, 'InputSignal')
num = (real_T*2).in_dll(dll,  "num")
den = (real_T*2).in_dll(dll,  "den")

In [ ]:
OutputSignal = real_T.in_dll(dll, 'OutputSignal')
SimTime = real_T.in_dll(dll, 'SimTime')

Validate Matlab `c2d` match Python's `control` library's implementation

In [ ]:
import control
import numpy as np
Ts=1e-3
sys = control.TransferFunction([3], [2, 1])
sysd = control.c2d(sys, Ts)
sysd

In [ ]:
num[1]

In [ ]:
np.isclose(num[1], sysd.num)

In [ ]:
den[0]

In [ ]:
np.isclose(den[:], sysd.den)

# Running The Model.

Run the model and store the step, input, output and simulation time to a pandas dataframe.

In [ ]:
model_initialize();
rows = list()
for step in range(1000):
    row_tmp = {
    'step': model_step(),
    'time': float(SimTime.value),
    'input': float(InputSignal.value),
    'output': float(OutputSignal.value),
    }
    rows.append(row_tmp)

In [ ]:
df = pd.DataFrame(rows)
df

# Running The Model Varying Inputs

Generate a step response to test the transfer function.

In [ ]:
model_initialize();
InputSignal.value=0.0;
rows = list()
for step in range(10000):
    if step>=1000:
        InputSignal.value=1.0
    else:
        InputSignal.value=0.0
    row_tmp = {
    'step': model_step(),
    'time': float(SimTime.value),
    'input': float(InputSignal.value),
    'output': float(OutputSignal.value),
    }
    rows.append(row_tmp)
df = pd.DataFrame(rows)
df

In [ ]:
df.plot(x='time', y="output")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
control.bode(sysd);

# Sinewave Input

Generate a sinewave with ``numpy``. Plot the input and output of the transfer function with ``pandas``.

In [ ]:
model_initialize();
rows = list()
for step in range(int(10*1e3)):
    InputSignal.value=np.sin(2*np.pi*step/1e3)
    row_tmp = {
    'step': model_step(),
    'time': float(SimTime.value),
    'input': float(InputSignal.value),
    'output': float(OutputSignal.value),
    }
    rows.append(row_tmp)
df = pd.DataFrame(rows)
df.plot(x='time', y=["input", "output"])